In [22]:
import QuantLib as ql
import numpy as np
import scipy.stats as stats
import scipy.optimize as spo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from random import sample
import sklearn.neural_network as sknn
import matplotlib.pyplot as plt
import time
from scipy.integrate import *
import cmath as cpx
import math as m
from sklearn.metrics import mean_squared_error

In [2]:
def MakeUniformRandParams(ArrayMin,ArrayMax,nRand):
    if (len(ArrayMin)!=len(ArrayMax)):
        raise Exception('ArrayMax and ArrayMin must have the same size')
    else:
        nParams = len(ArrayMin)
        rand_seq = ql.UniformRandomSequenceGenerator(nParams*nRand,ql.UniformRandomGenerator())
        X = rand_seq.nextSequence().value()
        #X = [ArrayMin[j%nParams]+(ArrayMax[j%nParams]-ArrayMin[j%nParams])*X[j] for j in range(nParams*nRand)]
        res = np.zeros((nRand,nParams))
        LocalMax = 0
        res__ = 0
        res_ = 0
        for i in range(nRand):
            for j in range(nParams):
                AdjMax = ArrayMax[j]
                if j==2:
                    AdjMax = XiFellerAdjust(res__,res_,ArrayMax[j])
                res[i,j]=ArrayMin[j]+(AdjMax-ArrayMin[j])*X[i*nParams+j]
                res__ = res_
                res_ = res[i,j]
        #res = np.reshape(X,(nRand,nParams))
        return res

def CountNoFellerCondition(X):
    k=0
    for j in range(len(X)):
        if (X[j,2]>XiFellerMax(X[j,0],X[j,1])):
            k=k+1
    return k

def XiFellerMax(kappa,theta):
    return m.sqrt(2*kappa*theta)

def XiFellerAdjust(kappa,theta,xi_init):
    if (xi_init>XiFellerMax(kappa,theta)):
        return XiFellerMax(kappa,theta)
    return xi_init
    
ArrayMin = [0.1, 0.1,  0.01, -0.999, 0.1]
ArrayMax = [1.5, 1.5,  0.75,  0.999, 1.5]
nRand = 100000
start = time.time()
res=MakeUniformRandParams(ArrayMin,ArrayMax,nRand)
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')
print('No Feller Condition ' + str(CountNoFellerCondition(res)))

Total computation time = 0.44s
No Feller Condition 0


In [3]:
# GLOBAL CONFIG #
AsOfDate = ql.Date(15,10,2018)
ql.Settings.instance().evaluationDate = AsOfDate
DayCount = ql.Actual365Fixed()
Calendar = ql.UnitedStates()

In [31]:
# EUROPEAN OPTION CONFIG #
MaturityDates = []
strikes = []
exercises = []
payoffs = []
option_type = ql.Option.Call
ms = [1,2,3,6,0,6,0,0]
ys = [0,0,0,0,1,1,2,3]
for k in range(len(ms)):
    if (10+ms[k]>12):
        ys_sup=1
    else:
        ys_sup=0
    MaturityDates.append(ql.Date(15,(10+ms[k]-1)%12+1,2018+ys[k]+ys_sup))
    exercises.append(ql.EuropeanExercise(MaturityDates[k]))
for k in range(7):
        strikes.append(70+k*10)
        #payoffs.append(ql.PlainVanillaPayoff(option_type, strikes[k]))
print(MaturityDates)
print(strikes)
EuropeanOptions = []
for T in range(len(MaturityDates)):
    for K in range(len(strikes)):
        EuropeanOptions.append(ql.VanillaOption(ql.PlainVanillaPayoff(option_type,100-(100-strikes[K])*m.sqrt((MaturityDates[T]-AsOfDate)/365)), exercises[T]))
#print(EuropeanOptions)

[Date(15,11,2018), Date(15,12,2018), Date(15,1,2019), Date(15,4,2019), Date(15,10,2019), Date(15,4,2020), Date(15,10,2020), Date(15,10,2021)]
[70, 80, 90, 100, 110, 120, 130]


In [137]:
def d1(S0, K, r, sigma, T):
    m = S0/(K*np.exp(-r*T))
    return np.log(m)*(1/(sigma*np.sqrt(T)))+sigma*np.sqrt(T)/2

def d2(S0, K, r, sigma, T):
    return d1(S0,K,r,sigma,T)-sigma*np.sqrt(T)
 
def BS_CallPrice(S0,K,r,sigma,T):
    if T==0:
        return np.maximum(S0-K,0)
    else:
        return S0*stats.norm.cdf(d1(S0,K,r,sigma,T))-K*np.exp(-r*T)*stats.norm.cdf(d2(S0,K,r,sigma,T))

def Heston_CharacteristicFunctionLogPrice(t,omega,S0,v0,r,kappa,theta,ksi,rho):
    alpha = complex(-omega**2 / 2, -omega / 2)
    beta = complex(kappa,-rho*ksi*omega)
    gamma = ksi**2 / 2
    h = (beta**2-4*alpha*gamma)**0.5
    r_m = (beta-h) / ksi**2
    r_p = (beta+h) / ksi**2
    g = r_m / r_p
    C = kappa*(r_m*t-2*cpx.log((1-g*cpx.exp(-h*t))/(1-g))/ksi**2)
    D = r_m*((1-cpx.exp(-h*t))/(1-g*cpx.exp(-h*t)))
    return cpx.exp(complex(C*theta+D*v0,omega*np.log(S0*np.exp(r*t))))

def RPI1_HestonChFuncLnS(omega,K,t,S0,v0,r,kappa,theta,ksi,rho):
    num = Heston_CharacteristicFunctionLogPrice(t,omega-complex(0,1),S0,v0,r,kappa,theta,ksi,rho)*cpx.exp(complex(0,-omega*np.log(K)))
    denom = complex(0,omega*Heston_CharacteristicFunctionLogPrice(t,-complex(0,1),S0,v0,r,kappa,theta,ksi,rho))
    return (num / denom).real
def RPI2_HestonChFuncLnS(omega,K,t,S0,v0,r,kappa,theta,ksi,rho):
    num = Heston_CharacteristicFunctionLogPrice(t,omega,S0,v0,r,kappa,theta,ksi,rho)*cpx.exp(complex(0,-omega*np.log(K)))
    denom = complex(0,omega)
    return (num / denom).real
def Heston_CallPrice(K,S0,r,T,v0,kappa,theta,ksi,rho):
    Pi_1 = 1/2+1/np.pi*quad(RPI1_HestonChFuncLnS,0,np.inf,args=(K,T,S0,v0,r,kappa,theta,ksi,rho))[0]
    Pi_2 = 1/2+1/np.pi*quad(RPI2_HestonChFuncLnS,0,np.inf,args=(K,T,S0,v0,r,kappa,theta,ksi,rho))[0]
    return S0*Pi_1-K*np.exp(-r*T)*Pi_2

def IVol(Mkt,S0,K,r,T,vol0=0.1):
    def f(vol,Mkt_,S0_,K_,r_,T_,type_): return (BS_CallPrice(S0_,K_,r_,vol,T_)-Mkt)**2
    return spo.minimize(f,vol0,args=(Mkt,S0,K,r,T,type),tol=1.0e-10).x

In [32]:
# EUROPEAN OPTION CONFIG V2 - necessary in V1 for numeric values #
Expiries = [0.0833,0.1667,0.25,0.5,1,1.5,2,3]
BaseStrikes = [70,80,90,100,110,120,130]
Strikes = []
for T in range(len(Expiries)):
    for K in range(len(BaseStrikes)):
        Strikes.append(100-(100-BaseStrikes[K])*m.sqrt(Expiries[T])) 

In [33]:
# MARKET DATA
r=ql.SimpleQuote(0.01)
r_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(r),DayCount)
q=ql.SimpleQuote(0.0)
q_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(q),DayCount)
s0=ql.SimpleQuote(100)

# BLACK SCHOLES ENGINE ONLY USED FOR COMPUTING IMPLIED VOLATILITIES
vol_bs = ql.SimpleQuote(0.10)
vol_bs_ts = ql.BlackConstantVol(0,Calendar,ql.QuoteHandle(vol_bs),DayCount)
BSProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(s0),ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                               ql.BlackVolTermStructureHandle(vol_bs_ts))

# HESTON ENGINE
v0 = 0.01
kappa=0.5
theta=0.01
sigma=0.0001
rho=-0.5
HestonParams = [theta,kappa,sigma,rho,v0]
relTolerance=0.01
maxEval=10000
HestonProcess = ql.HestonProcess(ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                                 ql.QuoteHandle(s0),v0,kappa,theta,sigma,rho)
HestonModel = ql.HestonModel(HestonProcess)
HestonEngine = ql.AnalyticHestonEngine(HestonModel,relTolerance,maxEval)
for i in range(len(EuropeanOptions)):
    EuropeanOptions[i].setPricingEngine(HestonEngine)

In [77]:
# GENERATE DATAS
nRand = 50000
start = time.time()
MinParamsValues = [0.1,0.1,0.1,-0.999,0.1]
MaxParamsValues = [2.5,0.6,1.5,-0.10,0.6]
local_start = 0
local_end = 0
ArrParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)
intermediary = time.time()

#npv = np.zeros(nRand)
#iv = np.zeros(nRand)
#for i in range(nRand):
#    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
#    npv[i] = EuropeanOption.NPV()
#    iv[i] = EuropeanOption.impliedVolatility(npv[i],BSProcess,1.0e-4,200,1.0e-8,10)
#end = time.time()
#print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
#print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
#print('Total computation time = ' + str(round(end-start,2)) + 's')
#X = np.zeros((nRand*len(Strikes),len(ArrParams[0])+2))
npv = 0
iv = np.zeros((nRand,len(EuropeanOptions)))
for i in range(nRand):
    HestonModel.setParams([ArrParams[i,k] for k in range(len(MinParamsValues))])
    for j in range(len(EuropeanOptions)):
        npv = max(EuropeanOptions[j].NPV(),np.random.uniform(0,0.01))
        iv[i,j] = EuropeanOptions[j].impliedVolatility(npv,BSProcess,1.0e-4,200,1.0e-8,100)
        #X[i*len(EuropeanOptions)+j,0]=Expiries[int(j/len(Expiries))]
        #X[i*len(EuropeanOptions)+j,1]=Strikes[j]
        #X[i*len(EuropeanOptions)+j,2:] = ArrParams[i,:] 
end = time.time()
print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
print('Total computation time = ' + str(round(end-start,2)) + 's')

Random params simulation time = 0.31s
Prices computation time = 185.5s
Total computation time = 185.81s


In [148]:
# GENERATE DATAS V2
nRand = 10
start = time.time()
S0 = 100
r = 0.01
MinParamsValues = [0.1,0.1,0.1,-0.999,0.1]
MaxParamsValues = [2.5,0.5,1.5,-0.10,0.5]
local_start = 0
local_end = 0
ArrParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)
intermediary = time.time()

#npv = np.zeros(nRand)
#iv = np.zeros(nRand)
#for i in range(nRand):
#    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
#    npv[i] = EuropeanOption.NPV()
#    iv[i] = EuropeanOption.impliedVolatility(npv[i],BSProcess,1.0e-4,200,1.0e-8,10)
#end = time.time()
#print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
#print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
#print('Total computation time = ' + str(round(end-start,2)) + 's')
npv = np.zeros(nRand*len(Strikes))
iv = np.zeros(nRand*len(Strikes))
for i in range(nRand):
    kappa = ArrParams[i,0]
    theta = ArrParams[i,1]
    xi = ArrParams[i,2]
    rho = ArrParams[i,3]
    v0 = ArrParams[i,4]
    j = 0
    for T in range(len(Expiries)):
        for K in range(len(BaseStrikes)):
            npv[i*len(Strikes)+j] = Heston_CallPrice(Strikes[j],S0,r,Expiries[T],v0,kappa,theta,xi,rho)
            iv[i*len(Strikes)+j] = IVol(npv[i*len(Strikes)+j],S0,Strikes[j],r,Expiries[T])
            j = j+1
end = time.time()
print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
print('Total computation time = ' + str(round(end-start,2)) + 's')

Random params simulation time = 0.0s
Prices computation time = 14.46s
Total computation time = 14.46s


In [61]:
HestonPrice = EuropeanOptions[j].NPV()
time_=ql.Date(15,11,2018)-AsOfDate
print(time_)
print(EuropeanOptions[j])

31
<QuantLib.QuantLib.VanillaOption; proxy of <Swig Object of type 'VanillaOptionPtr *' at 0x000001D84F6E5810> >


In [63]:
print(HestonPrice)
print('K = ' + str(strikes[j%len(strikes)]))
print('T = ' + str(MaturityDates[int(j/len(strikes))]))
print(ArrParams[i])
new_vol = 0.1
vol_bs.setValue(new_vol)
EuropeanOptions[j].setPricingEngine(ql.AnalyticEuropeanEngine(BSProcess))
print(EuropeanOptions[j].NPV())

6.29454661912861
K = 70
T = January 15th, 2019
[ 0.32584872  0.4189331   0.4476348  -0.84699097  0.01126491]
14.810631930529823


In [20]:
def ann_layers_neurones(layer, neurone):
    hidden_layer_sizes = (neurone, )
    if layer == 1.0: return hidden_layer_sizes
    elif layer > 1.0:
        for i in range(layer-1):
            hidden_layer_sizes += (neurone,)
        return hidden_layer_sizes

nLayersMin = 3
nLayersMax = 5
nLayersStep = 1
nNeuronesMin = 30
nNeuronesMax = 60
nNeuronesStep = 5
hidden_layers_sizes_vect = []
i_ = 0
for i in range(nNeuronesMin,nNeuronesMax+1,nNeuronesStep):
    j_ = 0
    for j in range(nLayersMin,nLayersMax+1,nLayersStep):
        hidden_layers_sizes_vect.append(ann_layers_neurones(j,i))
        j_ += 1
    i_ += 1
print(hidden_layers_sizes_vect)

[(30, 30, 30), (30, 30, 30, 30), (30, 30, 30, 30, 30), (35, 35, 35), (35, 35, 35, 35), (35, 35, 35, 35, 35), (40, 40, 40), (40, 40, 40, 40), (40, 40, 40, 40, 40), (45, 45, 45), (45, 45, 45, 45), (45, 45, 45, 45, 45), (50, 50, 50), (50, 50, 50, 50), (50, 50, 50, 50, 50), (55, 55, 55), (55, 55, 55, 55), (55, 55, 55, 55, 55), (60, 60, 60), (60, 60, 60, 60), (60, 60, 60, 60, 60)]


In [78]:
start = time.time()
n = nRand
ann = sknn.MLPRegressor(hidden_layer_sizes=(30,30,30))
array_train_rel_error = np.zeros((n,len(EuropeanOptions)))
array_test_rel_error = np.zeros((n,len(EuropeanOptions)))
array_train_abs_error = np.zeros((n,len(EuropeanOptions)))
array_test_abs_error = np.zeros((n,len(EuropeanOptions)))
index = sample(range(0,nRand),n)
X=np.zeros((n,len(ArrParams[0])))
for k in range(len(ArrParams[0])):        
    X[:,k]=[ArrParams[j,k] for j in index]
Y=[iv[j,:] for j in index]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# ANN FIT

mse_optim = 1000
ann_optim = ann
for current_archi in hidden_layers_sizes_vect:
    ann.set_params(hidden_layer_sizes=current_archi)
    ann.fit(X_train,Y_train)
    train_prediction=ann.predict(X_train)
    mse=mean_squared_error(Y_train,train_prediction)
    if (mse<mse_optim):
        mse_optim=mse
        ann_optim=ann
        test_prediction=ann.predict(X_test)
        for i in range(max(len(X_train),len(X_test))):
            for j in range(len(EuropeanOptions)):
                if (i < len(X_train)):
                    array_train_abs_error[i,j]=(train_prediction[i,j]-Y_train[i][j])*100
                    array_train_rel_error[i,j]=(train_prediction[i,j]/Y_train[i][j]-1)*100
                if (i < len(X_test)):
                    array_test_abs_error[i,j]=(test_prediction[i,j]-Y_test[i][j])*100
                    array_test_rel_error[i,j]=(test_prediction[i,j]/Y_test[i][j]-1)*100
        #array_train_error=[[ann.predict(X_train)[i,j]-Y_train[i][j] for j in range(len(EuropeanOptions))] for i in range(n)]
        #array_test_error=[[ann.predict(X_test)[i,j]-Y_test[i][j] for j in range(len(EuropeanOptions))] for i in range(n)]
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')

Total computation time = 69.77s


In [79]:
mean_abs_train = np.mean(array_train_abs_error,axis=0)
mean_abs_test = np.mean(array_test_abs_error,axis=0)
std_abs_train = np.std(array_train_abs_error,axis=0)
std_abs_test = np.std(array_test_abs_error,axis=0)
max_abs_train = np.amax(np.absolute(array_train_abs_error),axis=0)
max_abs_test = np.amax(np.absolute(array_test_abs_error),axis=0)

mean_rel_train = np.mean(array_train_rel_error,axis=0)
mean_rel_test = np.mean(array_test_rel_error,axis=0)
std_rel_train = np.std(array_train_rel_error,axis=0)
std_rel_test = np.std(array_test_rel_error,axis=0)
max_rel_train = np.amax(np.absolute(array_train_rel_error),axis=0)
max_rel_test = np.amax(np.absolute(array_test_rel_error),axis=0)

mean_abs_abs_train=np.mean(np.absolute(array_train_abs_error),axis=0)
mean_abs_rel_train=np.mean(np.absolute(array_train_rel_error),axis=0)
mean_abs_abs_test=np.mean(np.absolute(array_test_abs_error),axis=0)
mean_abs_rel_test=np.mean(np.absolute(array_test_rel_error),axis=0)

print("mean_abs_abs_train = ")
print(mean_abs_abs_train)
print('\n')
print("mean_abs_rel_train = ")
print(mean_abs_rel_train)
print('\n')
print("mean_abs_abs_test = ")
print(mean_abs_abs_test)
print('\n')
print("mean_abs_rel_test = ")
print(mean_abs_rel_test)
print('\n')
print("mean_abs_train = ")
print(mean_abs_train)
print('\n')
print("mean_abs_test = ")
print(mean_abs_test)
print('\n')
print("std_abs_train = ")
print(std_abs_train)
print('\n')
print("std_abs_test = ")
print(std_abs_test)
print('\n')
print("max_abs_train = ")
print(max_train)
print('\n')
print("max_abs_test = ")
print(max_test)
print('\n')
print("mean_rel_train = ")
print(mean_rel_train)
print('\n')
print("mean_rel_test = ")
print(mean_rel_test)
print('\n')
print("std_rel_train = ")
print(std_rel_train)
print('\n')
print("std_rel_test = ")
print(std_rel_test)
print('\n')
print("max_rel_train = ")
print(max_rel_train)
print('\n')
print("max_rel_test = ")
print(max_rel_test)
print('\n')

mean_abs_abs_train = 
[0.56261423 0.45787722 0.47707884 0.52680517 0.56771364 0.5109628
 0.51920413 0.48967076 0.55890289 0.64391708 0.49500306 0.55092476
 0.55516156 0.61852384 0.27910445 0.50899841 0.38953239 0.46696725
 0.77175426 0.53374278 0.69878768 0.46108325 0.49457004 0.6308957
 0.51989914 0.53202475 0.51889352 0.61222221 0.68063155 0.40950382
 0.44547711 0.48243848 0.51318776 0.45262333 0.60776395 0.4930184
 0.48354761 0.51496624 0.47261573 0.58552059 0.49716408 0.54958911
 0.60711045 0.57316358 0.50589061 0.52675434 0.59969214 0.57519948
 0.67302605 0.64965903 0.6453424  0.59520639 0.57981596 0.66758934
 0.64075912 0.74426342]


mean_abs_rel_train = 
[0.97473365 0.81386088 0.86400737 0.92924818 1.05083267 0.93724301
 0.96878448 0.82078568 0.96487808 1.09233312 0.85579204 0.96214011
 0.99850092 1.13451276 2.79093947 0.84095823 0.66600836 0.79251266
 1.31069287 0.93009744 1.2472794  0.70431624 0.76795848 0.99741096
 0.83659093 0.89163289 0.87897471 1.0454172  0.963694   0.5966

In [51]:
nTest = 3000
X_test = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nTest)
X_testAfterScale = scaler.transform(X_test)
iv = np.zeros(nTest)
err = np.zeros(nTest)
prediction = ann.predict(X_testAfterScale)
for i in range(nTest):
    HestonModel.setParams([X_test[i,k] for k in range(len(MinParamsValues))])
    iv[i] = EuropeanOption.impliedVolatility(EuropeanOption.NPV(),BSProcess,1.0e-4,200,1.0e-8,10)
    err[i]=abs(iv[i]-prediction[i])
stats.describe(err)

DescribeResult(nobs=3000, minmax=(3.6815891468755524e-07, 0.060570118918586824), mean=0.0047184862438876644, variance=1.4214250562972097e-05, skewness=2.378469056236739, kurtosis=19.338542371363665)

8.426932725704573